In [94]:
from docplex.mp.model import Model

# 데이터 정의
risk_free = [[0.03], [0.015], [0.01]]  # 현금 수익률 (1년, 3년, 5년)
r = [
    [0.262, 0.03, 0.012, 0.127],  # 주식, 장기채권, 단기채권, 금 (1년)
    [0.1, 0.015, -0.002, 0.023],  # 3년
    [0.156, 0.02, 0.012, 0.095]   # 5년
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1년
    [0.176, 0.06, 0.022, 0.142],  # 3년
    [0.185, 0.07, 0.019, 0.145]   # 5년
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1년
    [0.239, 0.04, 0.054, 0.159],  # 3년
    [0.239, 0.05, 0.054, 0.181]   # 5년
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현금 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(5)]  # 주식, 장기채권, 단기채권, 금, 현금
    
    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax),                  # 금
        risk_free[year_idx][0] * (1 - income_tax)         # 현금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[x[4]]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[1단계: 예수금 비중, 리스크 헷지] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print(f"  - 현금 비중(예수금)은 투자금의 {cash_ratio:.2%} 로 설정합니다.")
        print("\n")
#         print(f"  - 변동성(Volatility): {portfolio_volatility_val:.2%}")
#         print(f"  - 최대 손실(MDD): {portfolio_mdd_val:.2%}")
#         print("-" * 50)
        return cash_ratio
    else:
        print(f"[1단계: 현금 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: 주식, 장기채권, 단기채권, 금
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax)                   # 금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[2단계: 리스크 헷지 후, 자산 분배 비중]")
        for i, asset_name in enumerate(["주식", "장기채권", "단기채권", "금"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - 최대 수익률 (Objective): {solution.objective_value:.2%}")
        print(f"  - 최대 허용 변동성(Volatility): {portfolio_volatility_val:.2%}")
        print(f"  - 최대 허용 손실(MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
        print("\n")
    else:
        print(f"[2단계: 나머지 자산 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=1, 중립형=0, 추구형=0
  - 현금 비중(예수금)은 투자금의 29.93% 로 설정합니다.


[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 45.98%
  - 장기채권: 0.00%
  - 단기채권: 24.09%
  - 금: 0.00%
  - 최대 수익률 (Objective): 10.38%
  - 최대 허용 변동성(Volatility): 7.22%
  - 최대 허용 손실(MDD): 5.12%
--------------------------------------------------


[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=1, 추구형=0
  - 현금 비중(예수금)은 투자금의 8.84% 로 설정합니다.


[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 81.90%
  - 장기채권: 0.00%
  - 단기채권: 9.26%
  - 금: 0.00%
  - 최대 수익률 (Objective): 18.14%
  - 최대 허용 변동성(Volatility): 12.21%
  - 최대 허용 손실(MDD): 7.30%
--------------------------------------------------


[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=0, 추구형=1
  - 현금 비중(예수금)은 투자금의 2.04% 로 설정합니다.


[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 95.66%
  - 장기채권: 0.00%
  - 단기채권: 2.30%
  - 금: 0.00%
  - 최대 수익률 (Objective): 21.11%
  - 최대 허용 변동성(Volatility): 14.11%
  - 최대 허용 손실(MDD): 8.06%
--------------------------------------------------


[1단계: 예수금 비중, 리스크 헷지] 기간: 3

In [93]:
from docplex.mp.model import Model

# 데이터 정의
risk_free = [[0.051], [0.023], [0.019]]  # 현금 수익률 (1년, 3년, 5년)
r = [
    [0.262, 0.03, 0.012, 0.127],  # 주식, 장기채권, 단기채권, 금 (1년)
    [0.1, 0.015, -0.002, 0.023],  # 3년
    [0.156, 0.02, 0.012, 0.095]   # 5년
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1년
    [0.176, 0.06, 0.022, 0.142],  # 3년
    [0.185, 0.07, 0.019, 0.145]   # 5년
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1년
    [0.239, 0.04, 0.054, 0.159],  # 3년
    [0.239, 0.05, 0.054, 0.181]   # 5년
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현금 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(5)]  # 주식, 장기채권, 단기채권, 금, 현금
    
    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax),                  # 금
        risk_free[year_idx][0] * (1 - income_tax)         # 현금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[x[]]
#         portfolio_volatility_val = portfolio_volatility.solution_value
#         portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[1단계: 예수금 비중, 리스크 헷지] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print(f"  - 현금 비중(예수금)은 투자금의 {cash_ratio:.2%} 로 설정합니다.")
        print("\n")
#         print(f"  - 변동성(Volatility): {portfolio_volatility_val:.2%}")
#         print(f"  - 최대 손실(MDD): {portfolio_mdd_val:.2%}")
#         print("-" * 50)
        return cash_ratio
    else:
        print(f"[1단계: 현금 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: 주식, 장기채권, 단기채권, 금
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax)                   # 금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[2단계: 리스크 헷지 후, 자산 분배 비중]")
        for i, asset_name in enumerate(["주식", "장기채권", "단기채권", "금"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - 최대 수익률 (Objective): {solution.objective_value:.2%}")
        print(f"  - 최대 허용 변동성(Volatility): {portfolio_volatility_val:.2%}")
        print(f"  - 최대 허용 손실(MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
        print("\n")
    else:
        print(f"[2단계: 나머지 자산 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


SyntaxError: invalid syntax (228914961.py, line 65)

In [3]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현금 수익률 (1년, 3년, 5년)
r = [
    [0.262, 0.03, 0.012, 0.127],  # 주식, 장기채권, 단기채권, 금 (1년)
    [0.1, 0.015, -0.002, 0.023],  # 3년
    [0.156, 0.02, 0.012, 0.095]   # 5년
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1년
    [0.176, 0.06, 0.022, 0.142],  # 3년
    [0.185, 0.07, 0.019, 0.145]   # 5년
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1년
    [0.239, 0.04, 0.054, 0.159],  # 3년
    [0.239, 0.05, 0.054, 0.181]   # 5년
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현금 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # 주식, 장기채권, 단기채권, 금
    c = m.continuous_var(name="cash", lb=0)  # 현금

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax),                  # 금
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현금
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[1단계: 예수금 비중, 리스크 헷지] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print(f"  - 현금 비중(예수금)은 투자금의 {cash_ratio:.2%} 로 설정합니다.\n")
        return cash_ratio
    else:
        print(f"[1단계: 현금 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: 주식, 장기채권, 단기채권, 금
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
#     # 무위험 자산 제약
#     cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
#     cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
#     if y1_val == 1:  # 안정형
#         m.add_constraint(cash_volatility <= 0.00000001 * cash_ratio, "Volatility_Stable")
#         m.add_constraint(cash_mdd <= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
#     elif y2_val == 1:  # 중립형
#         m.add_constraint(cash_volatility <= 0.00000001 * cash_ratio, "Volatility_Neutral")
#         m.add_constraint(cash_mdd <= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
#     elif y3_val == 1:  # 추구형
#         m.add_constraint(portfol_volatility <= 0.000000002 * cash_ratio, "Volatility_Aggressive")
#         m.add_constraint(portfolio_mdd <= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax)                   # 금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[2단계: 리스크 헷지 후, 자산 분배 비중]")
        for i, asset_name in enumerate(["주식", "장기채권", "단기채권", "금"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - 최대 수익률 (Objective): {solution.objective_value:.2%}")
        print(f"  - 최대 허용 변동성(Volatility): {portfolio_volatility_val:.2%}")
        print(f"  - 최대 허용 손실(MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[2단계: 나머지 자산 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)



[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=1, 중립형=0, 추구형=0
  - 현금 비중(예수금)은 투자금의 29.93% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 45.98%
  - 장기채권: 0.00%
  - 단기채권: 24.09%
  - 금: 0.00%
  - 최대 수익률 (Objective): 10.38%
  - 최대 허용 변동성(Volatility): 7.22%
  - 최대 허용 손실(MDD): 5.12%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=1, 추구형=0
  - 현금 비중(예수금)은 투자금의 8.84% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 81.90%
  - 장기채권: 0.00%
  - 단기채권: 9.26%
  - 금: 0.00%
  - 최대 수익률 (Objective): 18.14%
  - 최대 허용 변동성(Volatility): 12.21%
  - 최대 허용 손실(MDD): 7.30%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=0, 추구형=1
  - 현금 비중(예수금)은 투자금의 2.04% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 95.66%
  - 장기채권: 0.00%
  - 단기채권: 2.30%
  - 금: 0.00%
  - 최대 수익률 (Objective): 21.11%
  - 최대 허용 변동성(Volatility): 14.11%
  - 최대 허용 손실(MDD): 8.06%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 3년, 투자 성향:

In [6]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현금 수익률 (1년, 3년, 5년)
r = [
    [0.262, 0.028, 0.042, 0.127],  # 주식, 장기채권, 단기채권, 금 (1년)
    [0.1, -0.123, -0.002, 0.023],  # 3년
    [0.156, -0.019,0.012, 0.095]   # 5년
]
σ = [
    [0.147, 0.208, 0.025, 0.151],  # 1년
    [0.176, 0.169, 0.022, 0.142],  # 3년
    [0.185, 0.163, 0.019, 0.145]   # 5년
]
mdd = [
    [0.083, 0.201, 0.009, 0.072],  # 1년
    [0.239, 0.436, 0.054, 0.159],  # 3년
    [0.239, 0.476, 0.054, 0.181]   # 5년
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현금 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # 주식, 장기채권, 단기채권, 금
    c = m.continuous_var(name="cash", lb=0)  # 현금

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax),                  # 금
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현금
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[1단계: 예수금 비중, 리스크 헷지] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print(f"  - 현금 비중(예수금)은 투자금의 {cash_ratio:.2%} 로 설정합니다.\n")
        return cash_ratio
    else:
        print(f"[1단계: 현금 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: 주식, 장기채권, 단기채권, 금
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 무위험 자산 제약
    cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Stable")
        m.add_constraint(cash_mdd >= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Neutral")
        m.add_constraint(cash_mdd >= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(cash_volatility  >= 0.000000002 * cash_ratio, "Volatility_Aggressive")
        m.add_constraint(cash_mdd >= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax)                   # 금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[2단계: 리스크 헷지 후, 자산 분배 비중]")
        for i, asset_name in enumerate(["주식", "장기채권", "단기채권", "금"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - 최대 수익률 (Objective): {solution.objective_value:.2%}")
        print(f"  - 최대 허용 변동성(Volatility): {portfolio_volatility_val:.2%}")
        print(f"  - 최대 허용 손실(MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[2단계: 나머지 자산 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=1, 중립형=0, 추구형=0
  - 현금 비중(예수금)은 투자금의 29.93% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 44.80%
  - 장기채권: 0.00%
  - 단기채권: 25.27%
  - 금: 0.00%
  - 최대 수익률 (Objective): 10.77%
  - 최대 허용 변동성(Volatility): 7.22%
  - 최대 허용 손실(MDD): 3.95%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=1, 추구형=0
  - 현금 비중(예수금)은 투자금의 8.84% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 81.44%
  - 장기채권: 0.00%
  - 단기채권: 9.71%
  - 금: 0.00%
  - 최대 수익률 (Objective): 18.30%
  - 최대 허용 변동성(Volatility): 12.21%
  - 최대 허용 손실(MDD): 6.85%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=0, 추구형=1
  - 현금 비중(예수금)은 투자금의 2.04% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 95.55%
  - 장기채권: 0.00%
  - 단기채권: 2.41%
  - 금: 0.00%
  - 최대 수익률 (Objective): 21.14%
  - 최대 허용 변동성(Volatility): 14.11%
  - 최대 허용 손실(MDD): 7.95%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 3년, 투자 성향:

In [1]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현금 수익률 (1년, 3년, 5년)
r = [
    [0.262,-0.062, 0.042, 0.127],  # 주식, 장기채권, 단기채권, 금 (1년)
    [0.1, -0.053, -0.002, 0.023],  # 3년
    [0.156, 0.002,0.012, 0.095]   # 5년
]
σ = [
    [0.147, 0.099, 0.025, 0.151],  # 1년
    [0.176, 0.085, 0.022, 0.142],  # 3년
    [0.185, 0.077, 0.019, 0.145]   # 5년
]
mdd = [
    [0.083, 0.199, 0.009, 0.072],  # 1년
    [0.239, 0.217, 0.054, 0.159],  # 3년
    [0.239, 0.232, 0.054, 0.181]   # 5년
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현금 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # 주식, 장기채권, 단기채권, 금
    c = m.continuous_var(name="cash", lb=0)  # 현금

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax),                  # 금
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현금
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[1단계: 예수금 비중, 리스크 헷지] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print(f"  - 현금 비중(예수금)은 투자금의 {cash_ratio:.2%} 로 설정합니다.\n")
        return cash_ratio
    else:
        print(f"[1단계: 현금 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: 주식, 장기채권, 단기채권, 금
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 무위험 자산 제약
    cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Stable")
        m.add_constraint(cash_mdd >= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Neutral")
        m.add_constraint(cash_mdd >= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(cash_volatility  >= 0.000000002 * cash_ratio, "Volatility_Aggressive")
        m.add_constraint(cash_mdd >= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # 안정형
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # 중립형
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # 추구형
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # 주식
        r[year_idx][1] * (1 - income_tax),                # 장기채권
        r[year_idx][2] * (1 - income_tax),                # 단기채권
        r[year_idx][3] * (1 - corp_tax)                   # 금
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[2단계: 리스크 헷지 후, 자산 분배 비중]")
        for i, asset_name in enumerate(["주식", "장기채권", "단기채권", "금"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - 최대 수익률 (Objective): {solution.objective_value:.2%}")
        print(f"  - 최대 허용 변동성(Volatility): {portfolio_volatility_val:.2%}")
        print(f"  - 최대 허용 손실(MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[2단계: 나머지 자산 최적화 실패] 기간: {year}년, 투자 성향: 안정형={y1_val}, 중립형={y2_val}, 추구형={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=1, 중립형=0, 추구형=0
  - 현금 비중(예수금)은 투자금의 29.93% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 44.80%
  - 장기채권: 0.00%
  - 단기채권: 25.27%
  - 금: 0.00%
  - 최대 수익률 (Objective): 10.77%
  - 최대 허용 변동성(Volatility): 7.22%
  - 최대 허용 손실(MDD): 3.95%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=1, 추구형=0
  - 현금 비중(예수금)은 투자금의 8.84% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 81.44%
  - 장기채권: 0.00%
  - 단기채권: 9.71%
  - 금: 0.00%
  - 최대 수익률 (Objective): 18.30%
  - 최대 허용 변동성(Volatility): 12.21%
  - 최대 허용 손실(MDD): 6.85%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 1년, 투자 성향: 안정형=0, 중립형=0, 추구형=1
  - 현금 비중(예수금)은 투자금의 2.04% 로 설정합니다.

[2단계: 리스크 헷지 후, 자산 분배 비중]
  - 주식: 95.55%
  - 장기채권: 0.00%
  - 단기채권: 2.41%
  - 금: 0.00%
  - 최대 수익률 (Objective): 21.14%
  - 최대 허용 변동성(Volatility): 14.11%
  - 최대 허용 손실(MDD): 7.95%
--------------------------------------------------
[1단계: 예수금 비중, 리스크 헷지] 기간: 3년, 투자 성향: